In [1]:
%%capture
import os

# check if this is a colab notebook and clone the repo if it is
if 'COLAB_GPU' in os.environ:
    %cd /content/
    !git clone https://github.com/Enver-group/twitch-web-analytics
    %cd twitch-web-analytics/notebooks

!pip install -r requirements.txt

!pip install -e .

In [3]:
# Tambien es necesario settear los token de acceso a la API de twitch
# os.environ['TWITCH_CLIENT_ID'] = '<YOUR_CLIENT_ID>'
# os.environ['TWITCH_CLIENT_SECRET'] = '<YOUR_CLIENT_SECRET>'

In [2]:
# autoreload modules
%load_ext autoreload
%autoreload 2

import src
from src.user import User

In [55]:
root = User.from_name("ibai")
root_follows = root.follows
print(f"{root.name.capitalize()} follows {len(root_follows)} users")
users_that_root_follows = User.get_users(root_follows)

Ibai follows 155 users


In [58]:
import pandas as pd
df = pd.DataFrame([root]+users_that_root_follows).sort_values("view_count", ascending=False)
df

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
111,36029255,Riot Games,None,partner,"Welcome to the Riot Games channel, home of LoL...",en,VALORANT,1336366323,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-09-08T06:53:10Z,None
115,19571641,Ninja,None,partner,,en,Fortnite,557890927,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-01-16T04:31:20Z,None
46,71092938,xQcOW,None,partner,THE BEST AT ABSOLUTELY EVERYTHING. THE JUICER....,en,Just Chatting,452263202,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-09-12T23:50:05Z,None
0,83232866,ibai,None,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286994651,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20T16:47:56Z,"[35980866, 145908612, 205218019, 91136321, 524..."
142,22346597,LVPes,None,partner,,es,Special Events,227606914,https://static-cdn.jtvnw.net/jtv_user_pictures...,2011-05-11T10:25:51Z,None
...,...,...,...,...,...,...,...,...,...,...,...
90,145953617,Rencorosa,None,affiliate,"Soy Cristina, tengo 28 años, suelo jugar a GTA...",es,The Witness,3551,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-01-25T16:22:32Z,None
120,108460978,borhabibi,None,,,es,League of Legends,815,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-12-01T16:15:25Z,None
153,88572935,leprosyfoxtrot,None,,,es,League of Legends,153,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-04-14T20:23:12Z,None
126,93611458,ismax95,None,,,en,,63,https://static-cdn.jtvnw.net/user-default-pict...,2015-06-15T18:26:09Z,None


In [60]:
from src.data import make_data_from_root_user

# Obtiene datos de cada usuario del arbol empezando por Ibai y sus follows
# cada iteracion siguiente coge un usuario del arbol aleatorio para añadir sus follows 
# y asi hasta llegar a max_users o que se interrumpa manualmente. 
# Cada 50 iteraciones guarda el progreso actual en un archivo csv
make_data_from_root_user("rubius",output_filepath="../data/data.csv",max_users=500)

INFO:src.data.make_dataset:Collecting users from a tree of twitch users follows starting from the follows of the user: rubius
INFO:src.data.make_dataset:New users will be fetched until 500 users are reached or the program is manually interrupted.
INFO:src.data.make_dataset:The tree will be expanded randomly starting from the root.
INFO:src.data.make_dataset:Iteration 1 - 119 users have been retrieved until now.
INFO:src.data.make_dataset:writing dataset to file data/rubius.csv
INFO:src.data.make_dataset:max_users reached. 502 users were retrieved.
INFO:src.data.make_dataset:writing dataset to file before stopping...


In [53]:
from src.data import extract_num_followers_from_users_df, extract_follows_from_users_df

# Dataset obtained from the tree of user follows starting from rubius as root node/leaf
df = pd.read_csv("../data/data.csv",lineterminator='\n')

# Extract the ids of the follows of the top 1000 users in the dataset by view_count
df = extract_follows_from_users_df(df,only_top=1000)

# Extracting the num_followers of the top 1000 users of the dataset of the tree starting from rubius follows
new_df = extract_num_followers_from_users_df(df,only_top=1000)

new_df.to_csv("../data/data.csv",index=False)
new_df.sort_values("num_followers",ascending=False)

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,459331509,auronplay,None,partner,feliz pero no mucho,es,Minecraft,212184340,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03T14:02:49Z,"['210708721', '431460701', '77649106', '130065..."
1,30857876,FolagorLives,None,partner,"Youtuber y a veces, Streamer.Canal de Youtube:...",es,VALORANT,41828690,https://static-cdn.jtvnw.net/jtv_user_pictures...,2012-05-27T19:37:36Z,"['503014047', '69564524', '40909862', '6829471..."
2,218908170,oscarbrock,None,partner,"¡Soy Oscar, Creador de contenido para GIANTS y...",es,Just Chatting,658100,https://static-cdn.jtvnw.net/jtv_user_pictures...,2018-05-06T18:21:18Z,"['30504119', '669133702', '641202528', '434195..."
3,152633332,knekro,None,partner,None,es,League of Legends,52657908,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-04-07T16:28:21Z,"['472263491', '42206347', '74101321', '4146728..."
4,157055506,SdeSergi0,None,affiliate,None,es,Minecraft,3062788,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-05-16T13:47:31Z,"['141449939', '148302141', '83949294', '319196..."
...,...,...,...,...,...,...,...,...,...,...,...
4179,412571101,Davizosky,None,partner,Tu pechofrío de confianza y cada día el de más...,es,FIFA 22,558062,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-01-27T11:05:50Z,"[227640221, 713413502, 605777965, 488749087, 2..."
4180,213918014,srbum10,None,None,None,es,Apex Legends,720,https://static-cdn.jtvnw.net/user-default-pict...,2018-04-13T07:54:02Z,None
4181,157723094,Sannset_,None,partner,"🍑 Mido 1,53 y hago directos de pie aunque pare...",es,Grand Theft Auto V,2307743,https://static-cdn.jtvnw.net/jtv_user_pictures...,2017-05-23T15:47:59Z,"[27234966, 704590714, 236134090, 43672019, 125..."
4182,69143777,K1ngDanT,None,None,Call of duty player @followdragons @DaanT1,es,None,6715,https://static-cdn.jtvnw.net/jtv_user_pictures...,2014-08-16T15:54:09Z,None
